In [ ]:
data = pd.read_csv('lyrics.csv')
data.head()

In [ ]:
#sub_data = data[['songs', 'seg', 'lyricist']] #創建了一個新DataFrame，但因為欄數不多所以可以不用

#重置索引，獲得一個包含了lyricist和合併後的seg值的新Series，其中seg值以逗號隔開
output_series = data.groupby(['lyricist'])['seg'].apply(lambda x: ','.join(x)).reset_index()
output_series

In [ ]:
output_series['seg'] = output_series['seg'].str.replace("', '", " ") #將','轉換成空格
output_series

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer #一個用於文本特徵提取的類別，將文本轉換為詞頻向量或者二進制向量表示

docs = output_series['seg'].tolist() #轉換為Python列表
vec = CountVectorizer()
X = vec.fit_transform(docs) #轉換成詞頻矩陣
#X.toarray()將詞頻矩陣轉換為二維數組
#vec.get_feature_names()獲取詞彙表中的特徵名稱，然後使用這些特徵名稱作為列名創建DataFrame
df = pd.DataFrame(X.toarray(), columns=vec.get_feature_names())
df

In [ ]:
#df.T是df的轉置，將df的行和列交換以便矩陣乘法操作
#dot()是DataFrame對象的方法，用於執行矩陣乘法。將df.T和df進行矩陣乘法運算，得到的結果是一個新的DataFrame，表示詞頻矩陣的內積
#coocc是內積結果，代表了詞彙之間的共現性。是一個方形矩陣，每個元素表示對應詞彙在資料中的共現次數
coocc = df.T.dot(df)
coocc

In [ ]:
import plotly.express as px

fig = px.imshow(coocc)
fig.layout.height = 1000
fig.layout.width = 1000
fig.show()

In [ ]:
result = coocc.values.flatten() #將詞彙共現矩陣轉換為一個包含所有共現頻次的一維數組
len(result)

In [ ]:
df_describe = pd.DataFrame(result) #獲得數據的描述性統計信息，提供對數據分布的整體了解，幫助理解數據的集中趨勢、變異程度和分布形狀
df_describe.describe()

In [ ]:
px.histogram(df) #繪製直方圖，x軸表示數值範圍，y軸表示頻次或數量

In [ ]:
import numpy as np

filter_num_up = 8 #表示篩選的上下閾值
filter_num_down = 10
getid = coocc.index
important = pd.DataFrame(columns = getid).T #創建一個空的DataFrame(important)，索引為getid（即coocc的索引）
important['relation'] = '' #新增一欄
dropid = [] #空列表dropid用於存要刪除的索引

for i in range(len(coocc)):
    result = coocc[(coocc.iloc[i] > filter_num_up) & (coocc.iloc[i] < filter_num_down)].index.tolist()
    if(len(result) > 0):
        important['relation'].loc[getid[i]] = coocc[(coocc.iloc[i] > filter_num_up) & (coocc.iloc[i] < filter_num_down)].index.tolist()
        print(important['relation'].loc[getid[i]])
    else:
        dropid.append(i)
        important = important.drop([getid[i]])

In [ ]:
important

In [ ]:
np.asarray(getid[dropid]) #將getid中索引為dropid的元素轉換為NumPy數組

In [ ]:
subcoocc = coocc.drop(np.asarray(getid[dropid])) #刪除在getid[dropid]中對應的索引位置的行和列後得到新的DataFrame
subcoocc

In [ ]:
#subcoocc.columns返回了subcoocc DataFrame的所有列索引，subcoocc.columns[dropid]取得要刪除的列的索引位置
#axis=1是指定要按列進行刪除
#inplace=True表示在原地進行修改，即直接修改subcoocc DataFrame而不返回新的DataFrame
subcoocc.drop(subcoocc.columns[dropid], axis=1, inplace=True)
subcoocc

In [ ]:
from IPython.core.display import HTML #導入用於在Jupyter Notebook中顯示HTML內容的庫

sta = subcoocc.describe()
display(HTML(sta.to_html())) #to_html()方法將sta DataFrame轉換為HTML格式的字符串，然後display顯示

In [ ]:
fig = px.imshow(subcoocc)
fig.layout.height = 900
fig.layout.width = 900
fig.update_traces(zmin=0, zmax=10) #設定顏色變化的數字範圍
fig.show()

In [ ]:
display(HTML(important.to_html()))

In [ ]:
px.histogram(coocc)